# Setup

TO DO:
- Retrain all dense layers
- Run for a greater number of epochs
- Crop/Flip the data to give a greater number of training examples 
- Normalise the data
- Look at misclassifications for further ideas
- Different boat validation set

In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
import keras
import vgg16
import os
import numpy as np

from vgg16 import Vgg16
from glob import glob
from keras.layers.core import Dense

Using TensorFlow backend.


In [3]:
model = Vgg16()

In [4]:
#path = 'data/'
path = 'data/sample_fish/'

# Create Validation Set

In [ ]:
for sub_path in os.listdir(path+'/train/'):
    g = glob(path+'train/'+sub_path+'/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(int(len(g)*0.2)):
        os.rename(shuf[i], path+'valid/' + sub_path + '/' + shuf[i].split('/')[-1])

# Fit a Model

In [5]:
bs=64
noe=6

In [6]:
batches = model.get_batches(path+'train', batch_size=bs)
valid_batches = model.get_batches(path+'valid', batch_size=bs*2)

Found 11 images belonging to 8 classes.
Found 11 images belonging to 8 classes.


In [7]:
%autoreload
model.finetune(batches, 0.001)

In [8]:
model.fit(batches, valid_batches, nb_epoch=noe)

Epoch 1/6
11/11 [==============================] - 25s - loss: 3.7862 - acc: 0.2727 - val_loss: 4.3777 - val_acc: 0.4545
Epoch 2/6
11/11 [==============================] - 23s - loss: 4.5158 - acc: 0.4545 - val_loss: 1.5180 - val_acc: 0.7273
Epoch 3/6
11/11 [==============================] - 23s - loss: 2.6202 - acc: 0.7273 - val_loss: 1.6941 - val_acc: 0.7273
Epoch 4/6
11/11 [==============================] - 23s - loss: 1.1706 - acc: 0.9091 - val_loss: 1.2656 - val_acc: 0.6364
Epoch 5/6
11/11 [==============================] - 23s - loss: 1.3440 - acc: 0.7273 - val_loss: 0.0557 - val_acc: 1.0000
Epoch 6/6
11/11 [==============================] - 24s - loss: 0.3406 - acc: 0.9091 - val_loss: 5.1684e-04 - val_acc: 1.0000


In [ ]:
model.model.save_weights('finetune1.h5')

Now optimise the other dense layers

In [9]:
layers = model.model.layers
first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
for layer in layers[first_dense_idx:]: layer.trainable=True

In [10]:
model.fit(batches, valid_batches, 6)

Epoch 1/6
11/11 [==============================] - 25s - loss: 0.0664 - acc: 0.9091 - val_loss: 2.8936e-06 - val_acc: 1.0000
Epoch 2/6
11/11 [==============================] - 23s - loss: 0.1311 - acc: 0.9091 - val_loss: 2.2760e-04 - val_acc: 1.0000
Epoch 3/6
11/11 [==============================] - 23s - loss: 5.9281e-06 - acc: 1.0000 - val_loss: 5.2778e-06 - val_acc: 1.0000
Epoch 4/6
11/11 [==============================] - 23s - loss: 6.0961e-06 - acc: 1.0000 - val_loss: 6.6961e-05 - val_acc: 1.0000
Epoch 5/6
11/11 [==============================] - 24s - loss: 0.0727 - acc: 0.9091 - val_loss: 1.2734e-06 - val_acc: 1.0000
Epoch 6/6
11/11 [==============================] - 24s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 5.0152e-05 - val_acc: 1.0000


In [ ]:
model.model.save_weights('finetune2.h5')

# Write out Predictions

In [ ]:
batches, preds = model.test(path+'test_stg1', batch_size = bs*2)

In [ ]:
filenames = batches.filenames

with open("sample_sub.csv", "w") as f:
    f.write("image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n")
    for pred,fn in zip(preds, filenames):
        pred = ",".join(map(lambda x: str(x), pred))
        f.write(fn+","+pred+"\n")

# Analyse Results

In [ ]:
model.model.load_weights('finetune1.h5')

In [ ]:
batches.filenames